In [1]:
%matplotlib inline

import os
import glob
import sys

import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image

base_dir = os.getcwd()
sys.path.append(base_dir)
im_dir = os.path.join(base_dir, 'images/')

imagenet_mean = np.array([ 0.48501961,  0.45795686, 0.40760392 ]) # Imagenet Mean (RGB)

In [ ]:
sys.path.append(os.path.join(base_dir, 'texture_synthesis_pytorch'))

import utilities
import model
import optimize

import importlib
importlib.reload(utilities)
importlib.reload(model)
importlib.reload(optimize)


import collections.abc as container_abcs

from argparse import Namespace
args = Namespace(**{
    'checkpoint_every': 1,
    'n_steps': 100,
    'max_iter': 5,
    'lr': 1.0
})

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

target_image = utilities.preprocess_image(utilities.load_image(os.path.join(im_dir, 'pebbles.jpg')))

net = model.Model(os.path.join(base_dir, 'models', 'VGG19_normalized_avg_pool_pytorch'), device, target_image)

optimizer = optimize.Optimizer(net, args)
result = optimizer.optimize()

final_image = utilities.postprocess_image(result, utilities.load_image(os.path.join(im_dir, 'pebbles.jpg')))
final_image.save('output.png')

In [ ]:
sys.path.append(os.path.join(base_dir, 'texture_synthesis_pytorch'))

import utilities
import model
import optimize

import importlib
importlib.reload(utilities)
importlib.reload(model)
importlib.reload(optimize)

lehl_img = utilities.load_image(os.path.join(im_dir, 'lehl_face.jpg'))
lehl_pre = utilities.preprocess_image(lehl_img)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

target_image = utilities.preprocess_image(utilities.load_image(os.path.join(im_dir, 'circuit_board.jpg')))

net = model.Model(os.path.join(base_dir, 'models', 'VGG19_normalized_avg_pool_pytorch'), device, target_image)

optimizer = optimize.Optimizer(net, args, start_image=lehl_pre)
result = optimizer.optimize()

final_image = utilities.postprocess_image(result, utilities.load_image(os.path.join(im_dir, 'circuit_board.jpg')))
final_image.save('output_lehl_face.png')


In [ ]:
utilities.postprocess_image(lehl_pre, utilities.load_image(os.path.join(im_dir, 'circuit_board.jpg'))).save('lehl_face_initial.jpg')

for i, img in enumerate(optimizer.opt_images):
    img.save(f"lehl_circuit_{i}.jpg")

In [ ]:
(lehl_pre.min(), lehl_pre.max(), target_image.min(), target_image.max())

In [ ]:
inv_reparam_func = optimizer.get_inv_reparam_func(net.target_image)
dn_lehl = inv_reparam_func(lehl_pre.clamp(float(target_image.min()), float(target_image.max())))
rnd_pre = torch.randn_like(target_image)
dn_rand = inv_reparam_func(rnd_pre.clamp(float(target_image.min()), float(target_image.max())))


print('lehl', dn_lehl.min(), dn_lehl.max(), dn_lehl.mean(), dn_lehl.median(), dn_lehl.std())
print('random', dn_rand.min(), dn_rand.max(), dn_rand.mean(), dn_rand.median(), dn_rand.std())

print('target', target_image.min(), target_image.max(), target_image.mean(), target_image.median(), target_image.std())
print('lehl_pre', lehl_pre.min(), lehl_pre.max(), lehl_pre.mean(), lehl_pre.median(), lehl_pre.std())
print('rnd_pre', rnd_pre.min(), rnd_pre.max(), rnd_pre.mean(), rnd_pre.median(), rnd_pre.std())

lehl_pre2 = torch.div(lehl_pre, 2)
print('lehl_pre', lehl_pre2.min(), lehl_pre2.max(), lehl_pre2.mean(), lehl_pre2.median(), lehl_pre2.std())

dn_lehl2 = inv_reparam_func(lehl_pre2)
print('lehl', dn_lehl2.min(), dn_lehl2.max(), dn_lehl2.mean(), dn_lehl2.median(), dn_lehl2.std())